Setup.

In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

print(f"Tensorflow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

Tensorflow version: 2.3.0
Keras version: 2.4.0


Load the mnist database.

In [6]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
image_data = np.load("../data/image_data.npy")
image_data = image_data.reshape((-1, 28, 28))
label_data = np.load("../data/label_data.npy")
print("image_data shape", image_data.shape)

row_indices = np.random.permutation(image_data.shape[0])
train_index = int(0.8 * image_data.shape[0])

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = np.append(x_train, image_data[row_indices[:train_index], :], axis=0)
y_train = np.append(y_train, label_data[row_indices[:train_index]], axis=0)
x_test = np.append(x_test, image_data[row_indices[train_index:], :], axis=0)
y_test = np.append(y_test, label_data[row_indices[train_index:]], axis=0)

# Shuffle
shuffled_indices = np.random.permutation(x_train.shape[0])
x_train = x_train[shuffled_indices]
y_train = y_train[shuffled_indices]

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


image_data shape (5070, 28, 28)
x_train shape: (64056, 28, 28, 1)
64056 train samples
11014 test samples


Build the model

In [7]:
# model = keras.models.load_model("../model.h5")
# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 576)              

Train the model.

In [8]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[tensorboard_callback])

Epoch 1/15
451/451 [==============================] - 27s 60ms/step - loss: 0.3702 - accuracy: 0.8812 - val_loss: 0.0941 - val_accuracy: 0.9703
Epoch 2/15
451/451 [==============================] - 27s 60ms/step - loss: 0.1053 - accuracy: 0.9669 - val_loss: 0.0618 - val_accuracy: 0.9814
Epoch 3/15
451/451 [==============================] - 22s 48ms/step - loss: 0.0811 - accuracy: 0.9743 - val_loss: 0.0574 - val_accuracy: 0.9816
Epoch 4/15
451/451 [==============================] - 20s 45ms/step - loss: 0.0649 - accuracy: 0.9796 - val_loss: 0.0513 - val_accuracy: 0.9860
Epoch 5/15
451/451 [==============================] - 21s 46ms/step - loss: 0.0555 - accuracy: 0.9832 - val_loss: 0.0445 - val_accuracy: 0.9880
Epoch 6/15
451/451 [==============================] - 20s 45ms/step - loss: 0.0485 - accuracy: 0.9845 - val_loss: 0.0456 - val_accuracy: 0.9870
Epoch 7/15
451/451 [==============================] - 20s 44ms/step - loss: 0.0445 - accuracy: 0.9858 - val_loss: 0.0411 - val_accuracy:

In [11]:
score = model.evaluate(x_test, y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

345/345 [==============================] - 1s 3ms/step - loss: 0.0257 - accuracy: 0.9923
Test loss: 0.025668535381555557
Test accuracy: 0.9922825694084167


In [14]:
model.save("model.h5")
print("Saved to model.h5")

Saved to model.h5
